# Initial Set-Up

In [1]:
import pickle as pkl
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

# import geopandas as gpd

import numpy as np

import rasterio

from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
# from mpl_toolkits.basemap import Basemap


import seaborn as sns

import numpy as np

import scipy

from matplotlib.colors import LogNorm

from sklearn import preprocessing

import dask.dataframe as dd

import xclim


/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
hq = pd.read_csv("../../data/companyData/justHQs.csv") 


min_lat = hq.latitude.min()
max_lat = hq.latitude.max()

min_lon = hq.longitude.min()
max_lon = hq.longitude.max()

In [3]:
'''baselineName = '../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/mirocHQs_198199.csv'
baseline = pd.read_csv(baselineName)'''

"baselineName = '../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/mirocHQs_198199.csv'\nbaseline = pd.read_csv(baselineName)"

# Weather Data
## *this is how we process the cmip6Data


In [4]:
import netCDF4 as nc
import xarray as xr

In [5]:
# https://kpegion.github.io/Pangeo-at-AOES/examples/advanced-analysis.html

In [97]:
weather = 'pr'
scenario = 'ssp585'

file1 = '../../../../../../../Volumes/backup2/dissData/cmip6Data/proj/' + weather + '/' + weather + '_day_MIROC6_' + scenario + '_r*i1p1f1_gn_20150101-20241231.nc'
file2 = '../../../../../../../Volumes/backup2/dissData/cmip6Data/proj/' + weather + '/' + weather + '_day_MIROC6_' + scenario + '_r*i1p1f1_gn_20250101-20341231.nc'
file3 = '../../../../../../../Volumes/backup2/dissData/cmip6Data/proj/' + weather + '/' + weather + '_day_MIROC6_' + scenario + '_r*i1p1f1_gn_20350101-20441231.nc'

data1 = xr.open_mfdataset(file1,concat_dim=['ensemble'],combine='nested',decode_times=True)
data2 = xr.open_mfdataset(file2,concat_dim=['ensemble'],combine='nested',decode_times=True)
data3 = xr.open_mfdataset(file3,concat_dim=['ensemble'],combine='nested',decode_times=True)

data = xr.combine_by_coords([data1,data2,data3])

In [99]:
def getFutureData(weather,scenario):

    start = time.time()

    # weather = 'pr'


    file1 = '../../../../../../../Volumes/backup2/dissData/cmip6Data/proj/' + weather + '/' + weather + '_day_MIROC6_' + scenario + '_r*i1p1f1_gn_20150101-20241231.nc'
    file2 = '../../../../../../../Volumes/backup2/dissData/cmip6Data/proj/' + weather + '/' + weather + '_day_MIROC6_' + scenario + '_r*i1p1f1_gn_20250101-20341231.nc'
    file3 = '../../../../../../../Volumes/backup2/dissData/cmip6Data/proj/' + weather + '/' + weather + '_day_MIROC6_' + scenario + '_r*i1p1f1_gn_20350101-20441231.nc'

    data1 = xr.open_mfdataset(file1,concat_dim=['ensemble'],combine='nested',decode_times=True)
    data2 = xr.open_mfdataset(file2,concat_dim=['ensemble'],combine='nested',decode_times=True)
    data3 = xr.open_mfdataset(file3,concat_dim=['ensemble'],combine='nested',decode_times=True)

    data = xr.combine_by_coords([data1,data2,data3])

    data.coords['lon'] = (data.coords['lon'] + 180) % 360 - 180
    data = data.sortby(data.lon)

    data = data.sel(lat=slice(min_lat,max_lat), 
                    lon=slice(min_lon,max_lon),
                   time=slice('2019-01-01','2039-12-31'))

    data['time'] = pd.to_datetime(data.time.values.astype(str))
    
    print("starting to load")
    start = time.time()
    # data = data.compute(workers = 100)

    time.time() - start

    # , ,combine = 'by_coords')   #  nc.Dataset(file)

    print("ending the load: ", time.time() - start)

    return(data)

In [6]:
start = time.time()


file = '../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/pr/pr_day_MIROC6_historical_r*i1p1f1_gn_19800101-19891231.nc'
data1 = xr.open_mfdataset(file,concat_dim=['ensemble'],combine='nested',decode_times=True)



# , ,combine = 'by_coords')   #  nc.Dataset(file)

print(time.time() - start)

81.81979489326477


In [7]:
start = time.time()


file = '../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/pr/pr_day_MIROC6_historical_r*i1p1f1_gn_19900101-19991231.nc'
# file = '../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/pr/pr_day_MIROC6_historical_r*i1p1f1_gn_19800101-19891231.nc'
# data = xr.open_mfdataset(file,concat_dim=['ensemble'],combine='nested',decode_times=True)
data2 = xr.open_mfdataset(file,concat_dim=['ensemble'],combine='nested',decode_times=True)
# , ,combine = 'by_coords')   #  nc.Dataset(file)

print(time.time() - start)

88.12129926681519


In [ ]:
# this now for some reason seems to work
'''from xclim import ensembles
import glob

import xarray as xr

import xclim as xc
ens = ensembles.create_ensemble(glob.glob(file)).load()'''

In [49]:
data = xr.combine_by_coords([data1,data2])

print(time.time() - start)
data.coords['lon'] = (data.coords['lon'] + 180) % 360 - 180
print(time.time() - start)
data = data.sortby(data.lon)
print(time.time() - start)

data = data.sel(lat=slice(min_lat,max_lat), 
                lon=slice(min_lon,max_lon),
               time=slice('1981-01-01','1999-12-31'))


data['time'] = pd.to_datetime(data.time.values.astype(str))

start = time.time()
data = data.compute(workers = 100)

time.time() - start

46.66204476356506
46.66490173339844
46.67260694503784


# Summaries
Summarize some of the data here.

### average weather by coords across ensemble and across time

In [14]:
start       = time.time()


overall     = data.mean(dim = ['ensemble','lat','lon'])
overallMean = overall.pr.values
print(time.time() - start)


overallStd  = data.std(dim = ['ensemble','lat','lon'])
overallStd  = overallStd.pr.values
print(time.time() - start)

3.4700090885162354
7.061110019683838


In [15]:
outfile =  '../../data/companyData/miroc_historicalMean.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(overallMean, pickle_file)
    
outfile =  '../../data/companyData/miroc_historicalStdDev.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(overallStd, pickle_file)

In [ ]:
'''start       = time.time()


byCoord = data.mean(dim = ['ensemble','time'])
ensembleMeanByCoord = byCoord.pr.values

time.time() - start'''

## 95th percentile by ensemble
find percentile by quarter

In [59]:
def getHistData(weather):

    start = time.time()

    # weather = 'pr'


    file1 = '../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/' + weather + '/' + weather + '_day_MIROC6_historical_r*i1p1f1_gn_19800101-19891231.nc'
    file2 = '../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/' + weather + '/' + weather + '_day_MIROC6_historical_r*i1p1f1_gn_19900101-19991231.nc'


    data1 = xr.open_mfdataset(file1,concat_dim=['ensemble'],combine='nested',decode_times=True)
    data2 = xr.open_mfdataset(file2,concat_dim=['ensemble'],combine='nested',decode_times=True)

    data = xr.combine_by_coords([data1,data2])

    data.coords['lon'] = (data.coords['lon'] + 180) % 360 - 180
    data = data.sortby(data.lon)

    data = data.sel(lat=slice(min_lat,max_lat), 
                    lon=slice(min_lon,max_lon),
                   time=slice('1981-01-01','1999-12-31'))

    data['time'] = pd.to_datetime(data.time.values.astype(str))
    
    print("starting to load")
    start = time.time()
    data = data.compute(workers = 100)

    time.time() - start

    # , ,combine = 'by_coords')   #  nc.Dataset(file)

    print("ending the load: ", time.time() - start)

    return(data)

In [83]:
df.where(df.time.dt.quarter == 1,drop=True)

<xarray.Dataset>
Dimensions:    (bnds: 2, ensemble: 50, lat: 25, lon: 64, time: 1714)
Coordinates:
  * lat        (lat) float64 21.71 23.11 24.51 25.91 ... 51.13 52.53 53.93 55.33
  * lon        (lon) float64 -157.5 -156.1 -154.7 ... -71.72 -70.31 -68.91
  * time       (time) datetime64[ns] 1981-01-01T12:00:00 ... 1999-03-31T12:00:00
    quarter    (time) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
Dimensions without coordinates: bnds, ensemble
Data variables:
    time_bnds  (ensemble, time, bnds) datetime64[ns] 1981-01-01 ... 1999-04-01
    lat_bnds   (time, ensemble, lat, bnds) float64 21.01 22.41 ... 54.63 56.03
    lon_bnds   (time, ensemble, lon, bnds) float64 201.8 203.2 ... 290.4 291.8
    pr         (ensemble, time, lat, lon) float32 1.5386958e-06 ... 4.6878404e-07

In [86]:
def getPercentileByQuarter(quarter, df):
    perQuarter     = df.where(df.time.dt.quarter == quarter,drop=True)
    percPerQuarter = perQuarter.quantile(0.95, dim = ('time'))
    
    return(percPerQuarter)

In [ ]:
pr = getHistData('pr')

In [88]:
q1_pr = getPercentileByQuarter(1,pr)
q2_pr = getPercentileByQuarter(2,pr)
q3_pr = getPercentileByQuarter(3,pr)
q4_pr = getPercentileByQuarter(4,pr)

In [104]:
pr_future.ensemble

<xarray.DataArray 'ensemble' (ensemble: 21)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])
Dimensions without coordinates: ensemble

In [ ]:
pr_future = getFutureData('pr','ssp585')

# subtract them and find out where > 0, per quarter
pr_future.where(pr_future.time.dt.quarter == 1,drop=True) - q1_pr


## tasmax - future

In [106]:
tasmax_future = getFutureData('tasmax','ssp585')
tasmax_future['tasmax_F'] = 1.8*(tasmax_future['tasmax']-273) + 32


# subtract them and find out where > 0, per quarter
# tasmax_future.where(pr_future.time.dt.quarter == 1,drop=True) - q1_pr


starting to load
ending the load:  1.1920928955078125e-06


In [120]:
tasmax_future['tasmax_F'] = 1.8*(tasmax_future['tasmax']-273) + 32
tasmax_future['extreme']  = 1*(tasmax_future['tasmax_F'] > 90)
futureTempExtremes        = tasmax_future.groupby(pr_future.time.dt.quarter).mean(dim = ['ensemble','time'])

In [124]:
futureTempExtremes.extreme.data


dask.array<concatenate, shape=(4, 25, 64), dtype=float64, chunksize=(1, 25, 64), chunktype=numpy.ndarray>

In [47]:
start                         = time.time()
percentileByQuarterByEnsemble = pr.groupby(pr.time.dt.quarter).quantile(0.95, dim = ('time'))
# percentileByEnsemble        = data.quantile(0.95, dim = ('time'))
time.time() - start

21.77485179901123

alternatives: can do percentiles on a per-ensemble basis, too.

In [17]:
outfile =  '../../data/companyData/miroc_historical95s.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(percentileByEnsemble, pickle_file)

In [18]:
percentileByEnsemble

<xarray.Dataset>
Dimensions:    (bnds: 2, ensemble: 50, lat: 25, lon: 64)
Coordinates:
  * lat        (lat) float64 21.71 23.11 24.51 25.91 ... 51.13 52.53 53.93 55.33
  * lon        (lon) float64 -157.5 -156.1 -154.7 ... -71.72 -70.31 -68.91
    quantile   float64 0.95
Dimensions without coordinates: bnds, ensemble
Data variables:
    time_bnds  (ensemble, bnds) datetime64[ns] 1999-01-18T02:23:59.999999952 ... 1999-01-19T02:23:59.999999952
    lat_bnds   (ensemble, lat, bnds) float64 21.01 22.41 22.41 ... 54.63 56.03
    lon_bnds   (ensemble, lon, bnds) float64 201.8 203.2 203.2 ... 290.4 291.8
    pr         (ensemble, lat, lon) float64 6.968e-05 7.182e-05 ... 0.0001168

In [20]:
data - percentileByEnsemble

<xarray.Dataset>
Dimensions:    (bnds: 2, ensemble: 50, lat: 25, lon: 64, time: 6939)
Coordinates:
  * lat        (lat) float64 21.71 23.11 24.51 25.91 ... 51.13 52.53 53.93 55.33
  * lon        (lon) float64 -157.5 -156.1 -154.7 ... -71.72 -70.31 -68.91
  * time       (time) datetime64[ns] 1981-01-01T12:00:00 ... 1999-12-31T12:00:00
    quantile   float64 0.95
Dimensions without coordinates: bnds, ensemble
Data variables:
    time_bnds  (ensemble, time, bnds) timedelta64[ns] -6592 days +21:36:00.000000048 ... 346 days 21:36:00.000000048
    lat_bnds   (time, ensemble, lat, bnds) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    lon_bnds   (time, ensemble, lon, bnds) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    pr         (ensemble, time, lat, lon) float64 -6.814e-05 ... -0.0001129

In [ ]:
file = open('../../data/companyData/miroc_historical95s.pkl','rb')
percentileByEnsemble = pkl.load(file)

In [ ]:
percentileByEnsemble

In [ ]:
# numpy and xarray and all that
# https://www.earthinversion.com/utilities/reading-NetCDF4-data-in-python/

In [ ]:
data.ensemble == 1

In [ ]:
# https://metview.readthedocs.io/en/latest/examples/ens_mean_spread_xarray.html
# data.mean(dim = 'ensemble')
overall = data.mean(dim = ['lat','lon'])
overall.pr.plot(hue = 'ensemble')

In [ ]:
plt.style.use("seaborn-dark")
plt.rcParams["figure.figsize"] = (13, 5)
ens.tas.plot(hue="realization")
plt.show()

In [ ]:
# this takes the ensemble mean and then the mean over time
data_emean = data.mean()# .mean() 
print(data_emean)

In [ ]:
import glob

import xarray as xr

import xclim as xc

# Set display to HTML sytle (for fancy output)
xr.set_options(display_style="html", display_width=50)

import matplotlib.pyplot as plt

%matplotlib inline

from xclim import ensembles

In [ ]:
len(glob.glob('../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/pr/pr_day_MIROC6_historical_r1*i1p1f1_gn_19800101-19891231.nc'))
datasets = glob.glob('../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/pr/pr_day_MIROC6_historical_r1*i1p1f1_gn_19800101-19891231.nc')
datasets

In [ ]:
# file = 
# data = xr.open_mfdataset(file,concat_dim='ensemble',combine='nested',decode_times=True)

ens = ensembles.create_ensemble(datasets, mf_flag = True)

In [ ]:
geoHQ = gpd.GeoDataFrame(
    allIG,
    geometry=gpd.points_from_xy(
        allIG["longitude"],
        allIG["latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

In [ ]:
one_point = data["pr"].sel(lat=50, lon=50)

In [ ]:
pd.DataFrame(test.temperature[test.year == 2010]).plot()

Now go through and make this iterable for the entire list.

In [ ]:
i = 0
data.pr.sel(allIG.iloc[0,:], method='nearest')

In [ ]:
hqPost2010 = geoHQ[geoHQ.archive_version_year > 2010].reset_index()

In [ ]:
start = time.time()



tempsList = list()
for i in range(0,hqPost2010.shape[0]):
    nearbyTemps = temps.sel(hqPost2010.loc[i,['latitude','longitude']], method='nearest')
    
    tempsList.append(list(np.array(nearbyTemps.temperature[nearbyTemps.year == hqPost2010.archive_version_year[i]])))

    if (i%100 == 0):
        print(i)
    
print(time.time() - start)






In [ ]:
outfile =  'data/hqDailyTemperatures.pkl'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(tempsList, pickle_file)

# Now let's do the percentiles

In [ ]:
baselineName = '../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/mirocHQs_198199.csv'
baseline = pd.read_csv(baselineName)

In [ ]:
baseline.s.unique()

In [ ]:
baseline[baseline.variable == 'tasmax_hist']

In [ ]:
quantsPrecip   = baseline[baseline.variable == 'pr_hist'][['zipcode','variable','quarter','value']].\
    groupby(['zipcode','variable','quarter']).quantile(.95).reset_index()
quantsPrecip.rename(columns = {'value': 'quantCutoff'}, inplace = True)
quantsPrecip['daysExtreme'] = 4.5


In [ ]:
quantsPrecip.head()

Find number of days above 90F.

In [ ]:
temp = baseline[baseline.variable == 'tasmax_hist'].drop(columns = {'s','year'})
temp.head()

In [ ]:
quantsTemp = baseline[baseline.variable == 'tasmax_hist'].drop(columns = {'s','year'})
quantsTemp['daysExtreme'] = 1*(quantsTemp.value > 305.3)
quantsTemp = quantsTemp.groupby(['zipcode', 'variable', 'quarter']).median().reset_index()
quantsTemp.rename(columns = {'value': 'quantCutoff'}, inplace = True)

quantsTemp.daysExtreme = quantsTemp.daysExtreme*90
quantsTemp.quantCutoff = 305.3


quantsTemp.head()

In [ ]:
quantsPrecip.head()

In [ ]:
quants = quantsPrecip.append(quantsTemp)

In [ ]:
quants.head()

And apply them to the next period.

In [ ]:
nextPeriodName = '../../../../../../../Volumes/backup2/dissData/cmip6Data/proj/mirocHQs_202040.csv'
nextPeriod = pd.read_csv(nextPeriodName)# .drop(columns = ['s'])

In [ ]:
nextPeriod.shape[0]/len(nextPeriod.zipcode.unique())/365/18/2/4

In [ ]:
nextPeriod.head()

In [ ]:
quants.daysExtreme

In [ ]:
nextPeriod = nextPeriod.merge(quants)
nextPeriod.head()

In [ ]:
nextPeriod['extreme'] = 1*(nextPeriod.value > nextPeriod.quantCutoff)

In [ ]:
# nextPeriod.to_csv("../../data/companyData/nextPeriodExtremes.csv")

In [ ]:
nextPeriod.head()

In [ ]:
nextPeriod.year

In [ ]:
nextPeriod['yearQuarter'] = nextPeriod.year.astype('int64') + nextPeriod.quarter.str[1:2].astype('int64')/4

In [ ]:
nextPeriod.head()

In [ ]:
nextPeriod[['zipcode']].drop_duplicates().shape

In [ ]:
nextPeriod.head()

In [ ]:
nextPeriod[nextPeriod.variable == 'pr_hist'].extreme.describe()

In [ ]:
quarterlyExtremes = nextPeriod[['variable','s','year','quarter','yearQuarter','zipcode','extreme']].\
    groupby(['variable','s','year','quarter','yearQuarter','zipcode']).sum().reset_index()

In [ ]:
zipLats = pd.read_csv("../../data/companyData/quarterlyExtremesMean.csv")[['zipcode','latitude','longitude']].\
    groupby(['zipcode']).mean().reset_index()

quarterlyExtremes = quarterlyExtremes.merge(zipLats)

In [ ]:
quarterlyExtremes.head()

In [ ]:
quarterlyExtremes.head()

In [ ]:
quarterlyExtremes.to_csv("../../data/companyData/quarterlyExtremesAll.csv")

In [ ]:
quarterlyExtremesMean = quarterlyExtremes[['variable','quarter','zipcode','latitude','longitude','extreme']].\
    groupby(['variable','quarter','zipcode','latitude','longitude']).mean().reset_index()

In [ ]:
quarterlyExtremesMean[quarterlyExtremesMean.variable == 'tasmax_hist'].extreme.describe()

In [ ]:
quarterlyExtremesMean[quarterlyExtremesMean.variable == 'pr_hist'].extreme.describe()

In [ ]:
quarterlyExtremesMean.head()

In [ ]:
quarterlyExtremesMean = quarterlyExtremesMean.merge(quants[['zipcode','variable','quarter','daysExtreme']])
quarterlyExtremesMean.head()

In [ ]:
quarterlyExtremesMean.to_csv("../../data/companyData/quarterlyExtremesMean.csv")

In [ ]:
zipLats.head()

In [ ]:
quarterlyExtremesMean.groupby(['variable']).extreme.describe()

------------------------------

In [ ]:
# we might have multiple records from within the same zipcode
quarterlyExtremesMean = quarterlyExtremes[['variable','quarter','zipcode','extreme']].\
    groupby(['variable','quarter','zipcode']).mean().reset_index()

# .groupby(['variable','quarter','zipcode']).mean().reset_index()


In [ ]:
quarterlyExtremesMean.groupby('variable').extreme.hist(bins = 100)

In [ ]:
(quarterlyExtremesMean[quarterlyExtremesMean.variable == 'tasmax_hist'].extreme - 4.5).hist(bins = 100)

In [ ]:
(quarterlyExtremesMean[quarterlyExtremesMean.variable == 'pr_hist'].extreme - 4.5).hist(bins = 100)

In [ ]:
sum(quarterlyExtremesMean[quarterlyExtremesMean.variable == 'pr_hist'].extreme > 4.5)/quarterlyExtremesMean[quarterlyExtremesMean.variable == 'pr_hist'].shape[0]

In [ ]:
sum(quarterlyExtremesMean[quarterlyExtremesMean.variable == 'tasmax_hist'].\
    extreme > 4.5)/quarterlyExtremesMean[quarterlyExtremesMean.variable == 'tasmax_hist'].shape[0]